In [1]:
from behavior_and_time_alignment import beh_and_time_alignment
from session_preprocessing import ephys_opto_preprocessing, session_crosscorr, ephys_opto_crosscorr
from opto_tagging import opto_plotting_session, opto_plotting_unit
from opto_waveforms_preprocessing import opto_wf_preprocessing
from drift_analysis import plot_session_opto_drift
import matplotlib.pyplot as plt
from utils.beh_functions import session_dirs, get_unit_tbl
import numpy as np
import pandas as pd
import os
from utils.plot_utils import combine_pdf_big
import spikeinterface as si
from joblib import Parallel, delayed
import pickle

In [2]:
file = '/root/capsule/scratch/684930/ecephys_684930_2023-09-27_10-04-04/ephys/opto/curated/ecephys_684930_2023-09-27_10-04-04_opto_waveform_metrics.pkl'
with open(file, 'rb') as f:
    wf_metrics = pickle.load(f)

In [3]:
wf_metrics

,unit_id,power,site,duration,pre_post,template,peak_channel,peak_waveform,spont,count,euclidean_norm,correlation
0,0,NaN,NaN,NaN,pre,"[[6.617517948150635, 6.7672810554504395, 10.75...",AP3,"[10.754636764526367, 10.398958206176758, 9.622...",1,250,NaN,NaN
1,1,NaN,NaN,NaN,pre,"[[3.906782388687134, -0.6104354858398438, 4.76...",AP1,"[3.906782388687134, 3.9678266048431396, 3.3166...",1,115,NaN,NaN
2,2,NaN,NaN,NaN,pre,"[[4.754879951477051, 5.840639591217041, 5.8406...",AP4,"[9.032401084899902, 11.044801712036133, 11.278...",1,250,NaN,NaN
3,3,NaN,NaN,NaN,pre,"[[-17.75592803955078, -14.302083969116211, -17...",AP3,"[-17.840160369873047, -17.484487533569336, -17...",1,250,NaN,NaN
4,4,NaN,NaN,NaN,pre,"[[10.1080322265625, 9.187376022338867, 8.74623...",AP4,"[8.28590202331543, 8.938031196594238, 10.62589...",1,122,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2743,446,0.2,11,10,pre,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",AP89,"[11.231999397277832, 10.06199836730957, 12.869...",0,10,0.082287,0.995836
2744,447,0.2,11,10,pre,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",AP84,"[11.31639289855957, 14.078359603881836, 11.968...",0,61,0.379069,0.995592
2745,448,0.2,11,10,pre,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",AP74,"[5.030999660491943, 4.972499847412109, 4.97249...",0,40,0.172746,0.989162
2746,449,0.2,11,10,pre,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",AP31,"[1.508000135421753, 2.2879996299743652, 4.3160...",0,45,0.122741,0.986845


In [3]:
unit_tbl = get_unit_tbl('behavior_754897_2025-03-14_11-28-53', 'curated')

In [6]:
unit_tbl[unit_tbl['unit_id']==48]['isi_violations_ratio']

42    0.411369
Name: isi_violations_ratio, dtype: float64

In [5]:
unit_tbl.columns

Index(['unit_id', 'bl_max_p', 'p_max', 'p_mean', 'lat_max_p', 'lat_mean',
       'euc_max_p', 'corr_max_p', 'opto_pass', 'amp', 'peak', 'real_unit',
       'y_loc', 'pass_count', 'spike_times', 'ks_unit_id',
       'isi_violations_ratio', 'firing_rate', 'presence_ratio',
       'amplitude_cutoff', 'decoder_label', 'depth', 'waveform_mean',
       'waveform_sd', 'default_qc', 'LC_range_top', 'LC_range_bottom',
       'tagged_loc', 'tagged', 'amplitude_opt', 'peak_wf_opt', 'mat_wf_opt'],
      dtype='object')

In [2]:
session_assets = pd.read_csv('/root/capsule/code/data_management/session_assets.csv')
session_list = session_assets['session_id']
session_list[:17]

0     ecephys_713854_2024-03-05_12-01-40
1     ecephys_713854_2024-03-05_13-01-09
2     ecephys_713854_2024-03-05_13-31-20
3     ecephys_717120_2024-03-06_12-23-53
4     ecephys_717120_2024-03-06_12-54-27
5     ecephys_717120_2024-03-07_12-12-02
6     ecephys_713854_2024-03-08_14-54-25
7     ecephys_713854_2024-03-08_15-43-01
8     ecephys_713854_2024-03-08_16-20-33
9     ecephys_713854_2024-03-08_17-15-58
10    ecephys_684930_2023-09-27_10-04-04
11    ecephys_684930_2023-09-28_11-45-27
12    ecephys_684930_2023-09-28_12-44-15
13    ecephys_687697_2023-09-15_11-30-06
14    ecephys_687697_2023-09-15_12-36-06
15    ecephys_691893_2023-10-05_12-46-57
16    ecephys_691893_2023-10-06_13-48-18
Name: session_id, dtype: object

In [4]:
# combine opto pdfs
data_type = 'curated'
def process(session, data_type):
    pdf_dir = session_dirs(session)[f'opto_dir_fig_{data_type}']
    output_pdf = os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_png.pdf')
    if os.path.exists(pdf_dir) and not os.path.exists(output_pdf):
        print(session)
        combine_pdf_big(pdf_dir, output_pdf)
# Parallel(n_jobs=4)(delayed(process)(session, data_type) for session in session_list)
for session in session_list[48:]:
    try:
        process(session, data_type)
        plt.close('all')
    except:
        print(f'Failed to process {session}')

behavior_754897_2025-03-12_12-23-15
Processing 113 files in /root/capsule/scratch/754897/behavior_754897_2025-03-12_12-23-15/ephys/opto/raw/figures
Combined PDF saved as: /root/capsule/scratch/754897/behavior_754897_2025-03-12_12-23-15/ephys/opto/raw/behavior_754897_2025-03-12_12-23-15_opto_tagging_png.pdf
behavior_754897_2025-03-13_11-20-42
Processing 273 files in /root/capsule/scratch/754897/behavior_754897_2025-03-13_11-20-42/ephys/opto/raw/figures
Combined PDF saved as: /root/capsule/scratch/754897/behavior_754897_2025-03-13_11-20-42/ephys/opto/raw/behavior_754897_2025-03-13_11-20-42_opto_tagging_png.pdf
behavior_754897_2025-03-14_11-28-53
Processing 125 files in /root/capsule/scratch/754897/behavior_754897_2025-03-14_11-28-53/ephys/opto/raw/figures
Combined PDF saved as: /root/capsule/scratch/754897/behavior_754897_2025-03-14_11-28-53/ephys/opto/raw/behavior_754897_2025-03-14_11-28-53_opto_tagging_png.pdf
behavior_754897_2025-03-15_11-32-18
Processing 155 files in /root/capsule/sc

In [5]:
# from joblib import Parallel, delayed
# session_list = ['behavior_751181_2025-02-25_12-12-35',
#                 'behavior_751181_2025-02-26_11-51-19']
data_type = 'curated'
def process(session, data_type='raw'):
    plt.close('all')
    print(f'{session} start')
    # beh_and_time_alignment(session) # plot behavior, check alignment between behavior, sound, and ephys
    plt.close('all')
    ephys_opto_preprocessing(session, data_type, 'soma') 
    # # redo spiketimes if not aligned, make opto_table (laser time in nidaq); 
    # # generate a pkl file with opto_resp/lat to each condition
    # plt.close('all')
    # # ephys_opto_crosscorr(session, data_type)
    # # separate session into opto/beh, cal cross-correlation
    # plt.close('all')
    # opto_wf_preprocessing(session, data_type, 'soma', load_sorting_analyzer=True)
    # recal wf based on conditions. (pre/post, power), cal similarity, euclidean distance, correlation with spont
    # pay attention to difference early and late in the session
    
    # opto_tagging_df = opto_plotting_session(session, data_type, 'soma', plot=True, resp_thresh=0.3, lat_thresh=0.02, save=True)
    
    # # save as pickle
    # with open(os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_metrics.pkl'), 'wb') as f:
    #     opto_tagging_df.to_pickle(f)
    
    # if '751004' in session:
    #     plot_session_opto_drift(session, data_type, update_csv=False)
    # else:     
    #     plot_session_opto_drift(session, data_type, update_csv=True)
    print(f'{session} stop')
        

# Parallel(n_jobs=4)(delayed(process)(session, data_type = data_type) for session in session_list[46:])

for session in session_list[11:17]:
    print(session)
    session_dir = session_dirs(session)
    if session_dir['curated_dir_curated'] is not None:
        print('Curated')
        process(session, data_type='curated') 
    # elif  session_dir['curated_dir_raw'] is not None:
    #     print('Raw')
    #     process(session, data_type='raw')
        


/opt/conda/lib/python3.10/site-packages/spikeinterface/core/base.py:1109: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter metrics_kwargs from template_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy paramet

In [3]:
ephys_opto_preprocessing('ecephys_684930_2023-09-27_10-04-04', 'curated', 'soma')

ecephys_684930_2023-09-27_10-04-04


/opt/conda/lib/python3.10/site-packages/spikeinterface/core/base.py:1109: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter metrics_kwargs from template_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy paramet

In [ ]:
datatype = 'curated'
list_exclude = ['716325', '717121', '717259', '717263', '714116', '724172', '722832', '739977', '744779', '739970', '751181', '751766_2025-02-15_12-08-11']
def process(session, data_type):
    if not any(exclude in session for exclude in list_exclude):
        print(f'{session} start')
        try:
            plot_session_opto_drift(session, data_type, update_csv=True, plot=True)
        except:
            plot(f'{session} failed')   
Parallel(n_jobs=4)(delayed(process)(session, data_type = datatype) for session in session_list)

In [ ]:
for session in session_list:
    try:
        process(session, data_type)
    except:
        print(f'Failed to process {session}')

In [ ]:
opto_tagging_df = opto_plotting_session('behavior_751004_2024-12-19_11-50-37', 'curated', 'soma', plot=False, resp_thresh=0.4, lat_thresh=0.02)

In [ ]:
ephys_opto_preprocessing('behavior_717121_2024-06-15_10-00-58', 'curated', 'soma')

behavior_717121_2024-06-15_10-00-58


/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter sparsity from quality_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from

In [ ]:
opto_tagging_df = opto_plotting_session('behavior_717121_2024-06-15_10-00-58', 'curated', 'soma', plot=True, resp_thresh=0.4, lat_thresh=0.02)

/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter sparsity from quality_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from

In [ ]:
ephys_opto_crosscorr('behavior_717121_2024-06-15_10-00-58', 'curated')
opto_wf_preprocessing('behavior_717121_2024-06-15_10-00-58', 'curated', 'soma', load_sorting_analyzer=False)

/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter sparsity from quality_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from

compute_waveforms:   0%|          | 0/6396 [00:00<?, ?it/s]

ValueError: Folder already exists /root/capsule/scratch/717121/behavior_717121_2024-06-15_10-00-58/ephys/opto/curated/opto_waveforms.zarr

In [ ]:
pip install ipykernel

In [ ]:
pip install Ipykernel

In [ ]:

Parallel(n_jobs=3)(delayed(process)(session) for session in session_list)

In [ ]:
plt.close('all')
data_type = 'raw'
session = session_list[0]
opto_tagging_df = opto_plotting_session(session, data_type, 'soma', plot=True, resp_thresh=0.4, lat_thresh=0.02)
plt.close('all')
opto_tagged_spike_stability(session, data_type, 'soma', opto_tagging_df=opto_tagging_df)

/root/capsule/code/beh_ephys_analysis/opto_tagging.py:345: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')


95 out of 271 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:386: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')


Merging file 0 out of 271
Merging file 50 out of 271
Merging file 100 out of 271
Merging file 150 out of 271
Merging file 200 out of 271
Merging file 250 out of 271
PDF files in '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/raw/figures' merged into '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/raw/behavior_751004_2024-12-19_11-50-37_opto_tagging.pdf' successfully.
5 out of 271 units pass quality control and opto tagging


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:458: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  opto_tagging_df_focus = opto_tagging_df_sorted[(opto_tagging_df_sorted['opto_pass'] == True) & (opto_tagging_df['decoder_label'] != 'artifact') & (opto_tagging_df['decoder_label'] != 'noise')].copy()


In [ ]:
session_list = ['']

In [ ]:
for session in session_list:
    ephys_opto_preprocessing(session, 'curated', 'soma')
    opto_wf_preprocessing(session, 'curated', 'soma', load_sorting_analyzer=False)
    opto_plotting_session(session, 'curated', plot=True)
    plt.close('all')

In [ ]:
session = 'behavior_751004_2024-12-20_13-26-11'
data_type = 'raw'
target = 'soma'
target_unit_ids = [16]
# pass_df = opto_plotting_session(session, data_type, target, plot=True,  resp_thresh=0.5, lat_thresh=0.015, target_unit_ids=target_unit_ids)

In [ ]:
data_type = 'raw'
target = 'soma'
for session in session_list:
    combine_qc = opto_plotting_session(session, data_type, target, plot=True,  resp_thresh=0.5, lat_thresh=0.015)

In [ ]:
session_dir = session_dirs(session)
opto_wf_csv = os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_waveform_metrics.csv')
opto_wf = pd.read_csv(opto_wf_csv)

In [ ]:
pip3 install --no-cache-dir numpy

SyntaxError: invalid syntax (4263930493.py, line 1)